In [1]:
import os
import numpy as np
import pandas as pd
from pandas import DataFrame as df

# Load data
total_epodata=np.load('epoched_data.npy', allow_pickle=True)
ori_templet=np.load('ori_templet.npy', allow_pickle=True)

In [2]:
type=1 # type=1 is epoched by saccade, 2 is epoched by stimulus

final_cat=[]
data_cat=[]
val_cat=[]
dur_cat=[]
rejectedXY_cat=[]
for c_num in range(len(total_epodata)):

    temp_s=total_epodata[c_num,0]
    X=temp_s[0]; Y=temp_s[1]; TargetXY=[X[:,500],Y[:,500]]; tt=np.shape(X)
    
    cat_ori=np.zeros(tt[0])
    for a in range(len(ori_templet)):

        temp_ot=ori_templet[a,:]
        temp_ot=np.tile(temp_ot,np.size(TargetXY,axis=1))
        temp_ot=temp_ot.reshape(np.shape(TargetXY)[1],np.shape(TargetXY)[0])
        idx=np.round(TargetXY)==np.round(temp_ot.T)
        idx2=np.sum(idx, axis=0)
        idx3=idx2==2
        cat_ori[idx3]=a+1

    final_cat.append(cat_ori)

    # sorting data as orientation
    
    if type==1:
        TT=temp_s[3]
    elif type==2:
        TT=temp_s[5]

    VV=temp_s[2]
    DD=temp_s[4]

    idx_all = [cat_ori == i for i in range(9)]

    # spike data sorting
    data_catt=[TT[idx_all[1],:],
              TT[idx_all[2],:],
              TT[idx_all[3],:],
              TT[idx_all[4],:],
              TT[idx_all[5],:],
              TT[idx_all[6],:],
              TT[idx_all[7],:],
              TT[idx_all[8],:],
              TT[idx_all[0],:]
             ]
    data_cat.append(np.array(data_catt))

    # velocity data sorting
    val_catt=[VV[idx_all[1],:],
             VV[idx_all[2],:],
             VV[idx_all[3],:],
             VV[idx_all[4],:],
             VV[idx_all[5],:],
             VV[idx_all[6],:],
             VV[idx_all[7],:],
             VV[idx_all[8],:],
             VV[idx_all[0],:]
            ]
    val_cat.append(np.array(val_catt))

    # duration data sorting
    dur_catt=[DD[idx_all[1]],
             DD[idx_all[2]],
             DD[idx_all[3]],
             DD[idx_all[4]],
             DD[idx_all[5]],
             DD[idx_all[6]],
             DD[idx_all[7]],
             DD[idx_all[8]],
             DD[idx_all[0]]
            ]
    dur_cat.append(np.array(dur_catt))    

    # X, Y data for rejected data
    rejectedXY_catt=np.array([X[idx_all[0],:],Y[idx_all[0],:]])
    rejectedXY_cat.append(rejectedXY_catt)

C:\Users\Jisub\AppData\Local\Temp\ipykernel_3040\1571159430.py:49: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data_cat.append(np.array(data_catt))
C:\Users\Jisub\AppData\Local\Temp\ipykernel_3040\1571159430.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val_cat.append(np.array(val_catt))
C:\Users\Jisub\AppData\Local\Temp\ipykernel_3040\1571159430.py:75: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you me

In [3]:
# accumulate total cell of each orientation

datacat_total=[]
valcat_total=[]
datacat_num=[]
for a in range(8):

    temp=[]
    temp2=[]
    temp_num=[]
    for c_num in range(len(data_cat)):

        pre_temp=data_cat[c_num][a]
        pre_temp2=val_cat[c_num][a]

        temp.append(pre_temp)
        temp2.append(pre_temp2)
        temp_num.append(np.shape(pre_temp)[0])

    datacat_total.append(temp)
    valcat_total.append(temp2)
    datacat_num.append(temp_num)

In [20]:
np.shape(datacat_total)

(8, 67)

In [ ]:
%% Plotting total

win=20; %timewindow (ms), if sampling rate is 1000hz, win=10 is 10ms
b=[2,3,9,15,14,13,7,1];
label_ori={'Up','Up right', 'Right', 'Down right',...
    'Down', 'Down left', 'Left', 'Up left'};

pos_total=[];
peak=[];
for a=1:8;

    % plotting velocity
    subplot(6,3,b(a))
    vv=mean(valcat_total{a}); [~,I]=max(vv);
    peak(a,1)=I;
    plot(vv,'r')
    xticks([0:50:600])
    xticklabels([-300:50:300])
    ylabel('Eye velocity');

    % Add the text
    xLimits = xlim; % [xmin, xmax]
    yLimits = ylim; % [ymin, ymax]
    xText = xLimits(1) + 0.02 * (xLimits(2) - xLimits(1)); % Slightly right of xmin
    yText = yLimits(2) - 0.15 * (yLimits(2) - yLimits(1)); % Slightly below ymax
    text(xText, yText, label_ori{a}, 'FontSize', 10, 'FontWeight', 'bold', 'Color', 'k');

    hold on
    plot([300 300],[0 600],'k', 'LineWidth',1);
    hold off

    % plotting heatmap
    subplot(6,3,b(a)+3)
    set(gcf,'color','w')
    imagesc(datacat_total{a});
    colormap(flipud(gray))
    xlabel('Time(ms)');
    ylabel('Trials');

    % Plotting line graph
    s_temp=smoothdata(sum(datacat_total{a}),'gaussian',win);
    [~,I2]=max(s_temp);
    peak(a,2)=I2;
    
    %for matalb code
    yyaxis right;

    plot(s_temp,'r')
    hold on
    plot([300 300],[0 300],'k', 'LineWidth',1);
    %     ylim([0 300])
    ylabel('Spikes');

    xticks([0:50:600])
    xticklabels([-300:50:300])

    ax = gca;
    ax.YAxis(1).Color = 'k';
    ax.YAxis(2).Color = 'r';

    ax=gca;
    pos = get(ax, 'Position');
    pos_total=[pos_total;pos];

    hold off

end; clear pos

for a=1:8;

    pos=pos_total(a,:);

    pp=pos(2)+0.01;
    set(subplot(6,3,b(a)), 'Position', [pos(1), pp+0.13, pos(3), pos(4)])
    set(subplot(6,3,b(a)+3), 'Position', [pos(1), pp, pos(3), pos(4)])

end


% bust onset relative to saccade onset(ms)
onset_delay=peak(:,2)-peak(:,1);
subplot(6,3,[8,11])
plot(onset_delay,'k')
ylabel('Bust onset-saccade onset(ms)');
xticks([1:2:8])
xticklabels([0:90:270])
xlabel('Saccade direction({\circ})');
xlim([0 9])

pos=pos_total(3,:);
set(subplot(6,3,[8,11]), 'Position', [pos(1)-0.282, pos(2)+0.036, pos(3), pos(4)*2])

set(gcf,'OuterPosition', [0, 0, 2000, 1000])

tightfig

%% Plotting each

win=20; %timewindow (ms), if sampling rate is 1000hz, win=10 is 10ms
b=[2,3,9,15,14,13,7,1];
label_ori={'Up','Up right', 'Right', 'Down right',...
    'Down', 'Down left', 'Left', 'Up left'};

for data_num=1:size(data_cat,1);

    valcat_t=val_cat(data_num,:);
    datacat_t=data_cat(data_num,:);

    figure

    pos_total=[];
    peak=[];
    for a=1:8;

        % plotting velocity

        subplot(6,3,b(a))
        vv=mean(valcat_t{a}); [~,I]=max(vv);
        peak(a,1)=I;
        plot(vv,'r')
        xticks([0:50:600])
        xticklabels([-300:50:300])
        ylabel('Eye velocity');

        % Add the text
        xLimits = xlim; % [xmin, xmax]
        yLimits = ylim; % [ymin, ymax]
        xText = xLimits(1) + 0.02 * (xLimits(2) - xLimits(1)); % Slightly right of xmin
        yText = yLimits(2) - 0.15 * (yLimits(2) - yLimits(1)); % Slightly below ymax
        text(xText, yText, label_ori{a}, 'FontSize', 10, 'FontWeight', 'bold', 'Color', 'k');

        hold on
        plot([300 300],[0 600],'k', 'LineWidth',1);
        hold off

        % plotting heatmap
        subplot(6,3,b(a)+3)
        set(gcf,'color','w')
        imagesc(datacat_t{a});
        colormap(flipud(gray))
        xlabel('Time(ms)');
        ylabel('Trials');

        % Plotting line graph
        s_temp=smoothdata(sum(datacat_t{a}),'gaussian',win);
        [~,I2]=max(s_temp);
        peak(a,2)=I2;

        yyaxis right;

        plot(s_temp,'r')
        hold on
        plot([300 300],[0 max(s_temp)],'k', 'LineWidth',1);

        %     ylim([0 300])
        ylabel('Spikes');

        xticks([0:50:600])
        xticklabels([-300:50:300])

        % octave code
        yyaxis left;
        ax_left = gca;
        set(ax_left, 'ycolor', 'k');

        yyaxis right;
        ax_right = gca;
        set(ax_right, 'ycolor', 'r');
        
        % matlab code
##        ax = gca;
##        ax.YAxis(1).Color = 'k';
##        ax.YAxis(2).Color = 'r';
##
##        pos = get(ax, 'Position');
##        pos_total=[pos_total;pos];

        hold off

    end; clear pos

##    for a=1:8;
##
##        pos=pos_total(a,:);
##
##        pp=pos(2)+0.01;
##        set(subplot(6,3,b(a)), 'Position', [pos(1), pp+0.13, pos(3), pos(4)]);
##        set(subplot(6,3,b(a)+3), 'Position', [pos(1), pp, pos(3), pos(4)]);
##
##    end


    % bust onset relative to saccade onset(ms)
    onset_delay=peak(:,2)-peak(:,1);
    subplot(6,3,[8,11])
    plot(onset_delay,'k')
    ylabel('Bust onset-saccade onset(ms)');
    xticks([1:2:8])
    xticklabels([0:90:270])
    xlabel('Saccade direction({\circ})');
    xlim([0 9])
##
##    pos=pos_total(3,:);
##    set(subplot(6,3,[8,11]), 'Position', [pos(1)-0.282, pos(2)+0.036, pos(3), pos(4)*2]);
##
##    set(gcf,'OuterPosition', [0, 0, 2000, 1000])

    tightfig

end

%% Plotting bust onset relative to saccade onset(ms)_average

win=20; %timewindow (ms), if sampling rate is 1000hz, win=10 is 10ms
b=[2,3,9,15,14,13,7,1];
label_ori={'Up','Up right', 'Right', 'Down right',...
    'Down', 'Down left', 'Left', 'Up left'};

peak_all=[];
for data_num=1:size(data_cat,1);

    valcat_t=val_cat(data_num,:);
    datacat_t=data_cat(data_num,:);

    peak=[];
    for a=1:8;

        % Peak
        vv=mean(valcat_t{a}); [~,I]=max(vv);
        peak(a,1)=I;
        s_temp=smoothdata(sum(datacat_t{a}),'gaussian',win);
        [~,I2]=max(s_temp);
        peak(a,2)=I2;
    end

    peak_all=cat(3,peak_all,peak);

end

onset_delay=squeeze(peak_all(:,2,:)-peak_all(:,1,:));

SEM = std(onset_delay')/sqrt(length(onset_delay));
Avg= mean(onset_delay,2);

figure; errorbar(Avg,SEM,'k')

ylabel('Bust onset-saccade onset(ms)');
xticks([1:2:8])
xticklabels([0:90:270])
xlabel('Saccade direction({\circ})');
xlim([0 9])


%% Plotting bust onset relative to saccade onset(ms)_each

win=20; %timewindow (ms), if sampling rate is 1000hz, win=10 is 10ms
b=[2,3,9,15,14,13,7,1];
label_ori={'Up','Up right', 'Right', 'Down right',...
    'Down', 'Down left', 'Left', 'Up left'};

peak_all=[];
for data_num=1:size(data_cat,1);

    valcat_t=val_cat(data_num,:);
    datacat_t=data_cat(data_num,:);

    peak=[];
    for a=1:8;

        % Peak
        vv=mean(valcat_t{a}); [~,I]=max(vv);
        peak(a,1)=I;
        s_temp=smoothdata(sum(datacat_t{a}),'gaussian',win);
        [~,I2]=max(s_temp);
        peak(a,2)=I2;
    end

    peak_all=cat(3,peak_all,peak);

end

onset_delay=squeeze(peak_all(:,2,:)-peak_all(:,1,:));

fig=figure;
for a=1:size(onset_delay,2);

    subplot(8,9,a)
    plot(onset_delay(:,a),'k')
    %     ylabel('Bust onset-saccade onset(ms)');
    title_cell=sprintf("Cell %d",a);
    title(title_cell)
    xticks([1:2:8])
    xticklabels([0:90:270])
    xlabel('Saccade direction({\circ})');
    xlim([0 9])
end

han=axes(fig,'visible','off');
##han.YLabel.Visible='on';
ylabel(han,'Bust onset-saccade onset(ms)');